# <b>Подготовка</b>

## Начало

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('vacancies_2024.csv')
df

## Курс валют 

In [6]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta


BASE_URL = "http://www.cbr.ru/scripts/XML_daily.asp"

salary_currencies = {
    "BYR": "R01090",
    "USD": "R01235",
    "EUR": "R01239",
    "KZT": "R01335",
    "UAH": "R01720",
    "AZN": "R01020A",
    "KGS": "R01370",
    "UZS": "R01717",
    "GEL": "R01210",
}

start_date = datetime(2003, 1, 1)
end_date = datetime(2024, 11, 1)

current_date = start_date
dates = []
while current_date <= end_date:
    dates.append(current_date.strftime("%d/%m/%Y"))
    current_date += timedelta(days=32)
    current_date = current_date.replace(day=1)


def get_currency_rate(date):
    params = {"date_req": date}
    response = requests.get(BASE_URL, params=params)

    root = ET.fromstring(response.content)
    rowForDf = {"date": date}
    
    for valute in root.findall("Valute"):
        char_code = valute.find("CharCode").text
        vunitRate = valute.find("VunitRate").text 

        if char_code in salary_currencies.keys(): 
            rowForDf[char_code] = vunitRate.replace(",", ".")
            
    return rowForDf


all_data = []
for date in dates:
    rowForDf = get_currency_rate(date)
    all_data.append(rowForDf)

columns = ["date", "BYR", "USD", "EUR", "KZT", "UAH", "AZN", "KGS", "UZS", "GEL"]
currency_df = pd.DataFrame(all_data, columns=columns)

currency_df['date'] = pd.to_datetime(currency_df['date'], format="%d/%m/%Y").dt.strftime("%Y-%m")
currency_df.to_csv("currency.csv", index=False, encoding="utf-8")


## Фильтрация вакансий

In [30]:
#web developer ключевые слова:
keywords = [
    'web develop', 'веб разработчик', 'web разработчик', 'web programmer', 'web программист', 'веб программист', 
    'битрикс разработчик', 'bitrix разработчик', 'битрикс девелопер', 'drupal разработчик', 'cms разработчик', 
    'wordpress разработчик', 'wp разработчик', 'joomla', 'drupal developer', 'cms developer', 'wordpress developer', 
    'wp developer', 'joomla developer', 'frontend developer', 'backend developer', 'fullstack developer', 
    'фронтенд разработчик', 'бекенд разработчик', 'фулстек разработчик', 'frontend engineer', 'backend engineer', 
    'fullstack engineer', 'front-end developer', 'back-end developer', 'full-stack developer', 'front-end engineer', 
    'back-end engineer', 'full-stack engineer', 'веб инженер', 'site developer', 'сайт разработчик', 
    'инженер-программист', 'разработчик сайтов', 'developer web', 'web engineer', 
    'php developer', 'php разработчик', 'python веб разработчик', 'node.js developer', 'node.js разработчик', 
    'javascript developer', 'javascript разработчик', 'typescript developer', 'typescript разработчик', 
    'vue.js developer', 'vue.js разработчик', 'react developer', 'react разработчик', 
    'angular developer', 'angular разработчик', 'django developer', 'django разработчик', 
    'джанго разработчик', 'flask developer', 'flask разработчик', 'ларевел разработчик', 
    'laravel developer', 'laravel разработчик', 'symfony developer', 'symfony разработчик', 
    'саймфони разработчик', 'ruby on rails developer', 'ruby разработчик', 'rails developer', 
    'ror developer', 'ruby on rails разработчик', 'рубивонрейлс разработчик', 'react.js разработчик', 
    'react.js developer', 'next.js developer', 'next.js разработчик'
]

keywords_pattern = '|'.join(keywords)

In [ ]:
filtred_df = df[df['name'].str.contains(keywords_pattern, case=False)]
filtred_df

## Перевод деняг

In [ ]:
from datetime import datetime

df_currency = pd.read_csv('currency.csv', index_col='date')


def convert_to_rur(salary, salary_currency, published_at):
    if pd.isna(salary): 
        return None
    if salary_currency == 'RUR':
        return salary
    
    date = datetime.fromisoformat(published_at)
    date = date.strftime("%Y-%m")

    rate_that_day = df_currency.loc[date]
    
    if salary_currency in df_currency.columns:
        return salary * rate_that_day[salary_currency]


def get_avg_salary(salary_from, salary_to):
    if pd.isna(salary_from) and pd.isna(salary_to):
        return None
    elif pd.isna(salary_from):
        return salary_to
    elif pd.isna(salary_to):
        return salary_from
    else:
        return (salary_from + salary_to) / 2
    

filtred_df['salary'] = filtred_df.apply(lambda row: get_avg_salary(row['salary_from'], row['salary_to']), axis=1)
filtred_df['salary'] = filtred_df.apply(lambda row: convert_to_rur(row['salary'], row['salary_currency'], row['published_at']), axis=1)
# csv_merged = csv_merged.dropna(subset=['salary'])
csv_merged = filtred_df[['name', 'key_skills','salary', 'area_name', 'published_at']]

csv_merged

## Еще фильтрация

In [ ]:
filtred_df = filtred_df[filtred_df['salary'] <= 1_000_000]

# <b>Страница «Общая статистика»</b>

## Динамика уровня зарплат по годам

## Динамика количества вакансий по годам

## Уровень зарплат по городам

## Доля вакансий по городам

## ТОП-20 навыков по годам

# <b>Страница «Востребованность»</b>